In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-09-19 09:46 ajna         INFO     Configuração de log efetuada


2025-09-19 09:46:54,981 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204', '8931404')" # Redex da Localfrio/Movecta, Redex da Santos Brasil Clia e Redex da DPW/EMBRAPORT
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, placaSemirreboque, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento, vazioConteiner
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
    AND vazioConteiner = 0 
"""

# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento
# Este '0' indica que o conteiner chegou estufado (cheio) ao Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)

In [6]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [7]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [8]:
# SQL com JOIN para filtrar saidas do REDEX seguidas de entrada no Terminal Exportador (Operador portuário)
sql_saida_redex = f"""
SELECT DISTINCT
    c.id, c.placa, c.placaSemirreboque, c.numeroConteiner, c.vazioConteiner,
    c.codigoRecinto, c.cpfMotorista, c.nomeMotorista, c.dataHoraOcorrencia, c.direcao
FROM apirecintos_acessosveiculo AS c
JOIN apirecintos_acessosveiculo AS e
    ON e.numeroConteiner = c.numeroConteiner AND e.placaSemirreboque = c.placaSemirreboque
    AND e.dataHoraOcorrencia >= c.dataHoraOcorrencia
    AND e.direcao = 'E'
    AND e.operacao = 'C'
    AND (e.codigoRecinto NOT IN {LISTA_REDEX} AND e.codigoRecinto NOT LIKE {FILTRO_LIKE_REDEX})
WHERE 
    c.dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND c.operacao = 'C'
    AND c.direcao = 'S'
    AND (c.codigoRecinto IN {LISTA_REDEX} OR c.codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND c.numeroConteiner != ''
    AND c.vazioConteiner = 0
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento
# Este '0' indica que o conteiner saiu estufado (cheio) do Redex
# A tabela c. é a tabela de ACESSO DE SAÍDA do REDEX no API RECINTOS
# A tabela e. é a tabela de ACESSO DE ENTRADA no TERMINAL EXPORTADOR (Operador Portuário)
# A lógica é que o caminhão com conteiner SAI DO REDEX ANTES DE ENTRAR NO TERMINAL EXPORTADOR (por isso o maior igual)

In [9]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [10]:
df_entrada_operadores

,id,placa,placaSemirreboque,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,vazioConteiner
0,12311094,FRS2D82,AIK5928,TGBU5884933,8931318,2025-09-12 09:58:02,C,E,12014047000193,11945072806,MARCOS CESAR DE MENDONCA,,,0
1,12311099,JKW0A21,AFM6I55,HAMU1425468,8931318,2025-09-12 10:40:42,C,E,32256956000145,41511693851,DANIEL RAMOS CANALONGA,,,0
2,12311098,JXZ3B31,DPC9D07,NIDU6208862,8931318,2025-09-12 10:41:50,C,E,32256956000145,91898412553,JOSE SILVIO NASCIMENTO SANTOS,,,0
3,12311102,BSF2052,AOO2030,MSNU1517873,8931318,2025-09-12 10:46:45,C,E,01896217000194,36519003808,WESLLEY POMBAL TEIXEIRA,,,0
4,12311104,BWN6E31,MMS8544,MEDU4850432,8931318,2025-09-12 10:54:17,C,E,01896217000194,10848936817,JOSE BENJAMIN DA SILVA,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22117,12509770,GAZ2G71,FEI2861,TLLU1088728,8931404,2025-09-18 23:54:31,C,E,05457125000169,06544067822,NELSON GARCIA,,,0
22118,12509772,MMJ7B11,TKC9D10,YMMU6530047,8931404,2025-09-18 23:54:41,C,E,21425093000176,11746934441,SANDRO DOS SANTOS,,,0
22119,12509777,KEV2G30,AZI6172,UTCU4916825,8931404,2025-09-18 23:55:06,C,E,09621639000169,46867943829,IGOR RAMOS DOS SANTOS,35250923120865000197550010000016711319077991,,0
22120,12509790,FPX8C82,EPU9B33,CXRU1132985,8931404,2025-09-18 23:55:43,C,E,05457125000169,26113216802,EDSON JOÃ?O DA SILVA,,,0


In [11]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [12]:
df_saida_redex

,id,placa,placaSemirreboque,numeroConteiner,vazioConteiner,codigoRecinto,cpfMotorista,nomeMotorista,dataHoraOcorrencia,direcao
0,12379734,KQL2H54,CPJ3693,TXGU5818338,0,8932796,00851885497,LUIZ ANDRE DE OLIVEIRA,2025-09-12 09:57:49,S
1,12378825,ESU7E91,EPU9B34,SZLU9346255,0,8932778,02851685848,ANTONIO PEREIRA,2025-09-12 09:58:00,S
2,12379128,DPF3F50,TKV7A07,TXGU5739157,0,8932796,07008971869,MARCELO GOUVEIA PEIXOTO,2025-09-12 09:59:18,S
3,12379735,JRC1E69,TMF6B85,EGSU1409387,0,8932796,34747344829,THIAGO PINTO DA CRUZ,2025-09-12 10:00:26,S
4,12379651,EPU7D03,DPF3957,EGHU9531778,0,8932796,59411112404,FRANCISCO DE ASSIS DA SILVA,2025-09-12 10:02:25,S
...,...,...,...,...,...,...,...,...,...,...
960,12509257,TBS1H49,FEI5A66,MRKU7950396,0,8931404,33274427896,RAPHAEL DANTAS DOS SANTOS,2025-09-18 21:06:38,S
961,12510098,CLH7C10,BXJ0875,MRSU7931212,0,8932709,32164884892,DEIVIDI MENEZES DA ROCHA,2025-09-18 21:13:32,S
962,12511192,ESU7C97,EPU9A32,MSKU5444092,0,8932778,06549854806,ROMILDO DE JESUS PAULA,2025-09-18 21:15:00,S
963,12509331,FSG8D86,ESU7E79,HAMU4494009,0,8931404,13375269838,JOSIMAR DO ESPIRITO SANTO,2025-09-18 21:31:14,S


In [13]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placaSemirreboque', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

In [14]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [15]:
# df_saidas_entradas

In [16]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [17]:
mapear_ids_mongo(db['fs.files'], df_saidas_entradas)

Buscando ids no MongoDB:   0%|          | 0/951 [00:00<?, ?it/s]

Registros: 951, imagens: 386


In [18]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'placaSemirreboque': 'placa do semireboque', 'numeroConteiner': 'contêiner', 'vazioConteiner' : 'conteinervazio',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX','placaSemirreboque': 'placa do semireboque',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

,id_E_Operador,placa_E_Operador,placa do semireboque,contêiner,Operador,Entrada Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista_E_Operador,...,id_S_REDEX,placa_S_REDEX,vazioConteiner_S_REDEX,Redex,cpfMotorista_S_REDEX,nomeMotorista_S_REDEX,Saída REDEX,direcao_S_REDEX,transitTime,_id
0,12423418,MTC4J24,FAX1317,HMMU6543221,8931318,2025-09-16 09:34:07,C,E,12014047000193,31704711800,...,12434386,MTC4J24,0,8931404,31704711800,JOSE ADILSON BISPO DOS SANTOS,2025-09-16 08:30:51,S,1.054444,68c97640f614e4f4040efa19
1,12423435,CUD4902,FOE4388,HMMU6790052,8931318,2025-09-16 09:56:49,C,E,12014047000193,10430518803,...,12434419,CUD4902,0,8931404,10430518803,ALDO PASCOAL SOARES JUNIOR,2025-09-16 08:53:56,S,1.048056,68c97640f614e4f4040efa1e
2,12423469,EWJ9661,FTU5124,KOCU4524804,8931318,2025-09-16 11:19:54,C,E,12014047000193,90643925449,...,12434555,EWJ9661,0,8931404,90643925449,ANDRE TEIXEIRA DA SILVA SANTOS,2025-09-16 10:16:34,S,1.055556,68ca98be4b6fcba5bbb13a1d
3,12423491,FEI3I38,FUY9846,HMMU5519489,8931318,2025-09-16 11:54:34,C,E,12014047000193,17898192885,...,12434571,FEI3I38,0,8931404,17898192885,ANTONIO CARLOS BRAGA,2025-09-16 10:22:29,S,1.534722,68ca98be4b6fcba5bbb13a18
4,12423598,KJQ0F31,FUI4356,MOFU6706340,8931318,2025-09-16 17:01:37,C,E,12014047000193,08923508810,...,12435152,KJQ0F31,0,8931404,08923508810,PEDRO DE SOUZA,2025-09-16 16:02:10,S,0.990833,68ca990e8e8f95cb27bf8bac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,12506249,TMB2G32,FQI5300,CXRU1062825,8931359,2025-09-18 21:46:01,C,E,02916265009116,02390279651,...,12378112,SEV8H03,0,8932761,04253152317,RICHARDSON PEREIRA PENHA,2025-09-14 11:47:46,S,105.970833,68cd3e24203e944c5a21cdea
947,12506336,ESU7C97,EPU9A32,MSKU5444092,8931359,2025-09-18 22:02:12,C,E,05457125000169,06549854806,...,12511192,ESU7C97,0,8932778,06549854806,ROMILDO DE JESUS PAULA,2025-09-18 21:15:00,S,0.786667,None
948,12506354,COZ7I26,IAU2759,GCXU6447473,8931359,2025-09-18 22:04:33,C,E,58130089000786,30731891813,...,12509938,COZ7I26,0,8932709,30731891813,RODRIGO DE CARVALHO SANTOS,2025-09-18 21:34:07,S,0.507222,None
949,12506362,CLH7C10,BXJ0875,MRSU7931212,8931359,2025-09-18 22:06:26,C,E,58130089000786,32164884892,...,12510098,CLH7C10,0,8932709,32164884892,DEIVIDI MENEZES DA ROCHA,2025-09-18 21:13:32,S,0.881667,None


In [19]:
from IPython.display import display, HTML

def get_outliers(df):
    mean = df['transitTime'].mean()
    std = df['transitTime'].std()
    outliers = df[(df['transitTime'] < mean - 1 * std) | (df['transitTime'] > mean + 1 * std)]
    return outliers

def generate_html(_id):
    url = f"https://ajna1.rfoc.srf/virasana/file?_id={_id}"
    img = f"https://ajna1.rfoc.srf/virasana/imagens_cmap?_id={_id}"
    return f'<a href="{url}"><img src="{img}" width="200"></a>'
    
outliers = get_outliers(df_saidas_entradas)[['placa do semireboque', 'contêiner', 'Redex', 'Saída REDEX',
                                  'Operador', 'Entrada Operador', 'transitTime', '_id']]

outliers['html'] = outliers['_id'].apply(generate_html)
display(HTML(outliers.to_html(escape=False)))

,placa do semireboque,contêiner,Redex,Saída REDEX,Operador,Entrada Operador,transitTime,_id,html
272,DBM7C27,FANU3325433,8932796,2025-09-15 18:33:35,8931356,2025-09-16 15:51:24,21.296944,None,
273,DBM7C27,FANU3325433,8932796,2025-09-15 18:49:29,8931356,2025-09-16 15:51:24,21.031944,None,
410,CUL2G26,MRSU7808759,8931404,2025-09-16 21:08:49,8931356,2025-09-17 22:47:32,25.645278,None,
436,FIX9153,MRSU7277230,8931404,2025-09-16 11:57:25,8931356,2025-09-18 03:02:21,39.082222,None,
445,DCF5B41,MRSU5009537,8931404,2025-09-16 14:08:06,8931356,2025-09-18 03:58:54,37.846667,None,
475,BTZ9C84,MRSU4206070,8931404,2025-09-16 09:27:16,8931356,2025-09-18 07:52:03,46.413056,None,
506,BRQ6G50,HAMU1863211,8931404,2025-09-16 18:29:09,8931356,2025-09-18 11:47:13,41.301111,None,
519,GZV4C99,TGBU9888670,8931404,2025-09-17 02:38:49,8931356,2025-09-18 14:13:56,35.585278,None,
521,MLW2F35,HLBU1207960,8931404,2025-09-16 23:26:51,8931356,2025-09-18 14:23:02,38.936389,None,
544,CUE0G55,MRKU2581588,8931404,2025-09-16 22:10:31,8931356,2025-09-18 16:52:43,42.703333,None,


In [20]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

In [21]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx', index=False)

In [22]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placaSemirreboque', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 7) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placaSemirreboque', 'numeroConteiner'], right_on=['placaSemirreboque', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 7) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) 
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'placaSemirreboque', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

KeyError: "['codigoRecinto_pesagem', 'codigoRecinto_saida'] not in index"